In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.conv1 = nn.Conv2d(1, 8, 3, stride=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(8, 8*8, 3, stride=2)
        self.relu2 = nn.ReLU()

        self.head = nn.Sequential(
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(256, 128), 
            nn.Dropout(0.2),
            nn.Linear(128, 10) 
            # nn.Softmax()
        )
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        y = self.head(x)
        y = self.dequant(y)
        return y

def fuse_conv_and_bn(conv, bn):
    # Fuse convolution and batchnorm layers https://tehnokv.com/posts/fusing-batchnorm-and-conv/
    fusedconv = nn.Conv2d(conv.in_channels,
                          conv.out_channels,
                          kernel_size=conv.kernel_size,
                          stride=conv.stride,
                          padding=conv.padding,
                          groups=conv.groups,
                          bias=True).requires_grad_(False).to(conv.weight.device)

    # prepare filters
    w_conv = conv.weight.clone().view(conv.out_channels, -1)
    w_bn = torch.diag(bn.weight.div(torch.sqrt(bn.eps + bn.running_var)))
    fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))

    # prepare spatial bias
    b_conv = torch.zeros(conv.weight.size(0), device=conv.weight.device) if conv.bias is None else conv.bias
    b_bn = bn.bias - bn.weight.mul(bn.running_mean).div(torch.sqrt(bn.running_var + bn.eps))
    fusedconv.bias.copy_(torch.mm(w_bn, b_conv.reshape(-1, 1)).reshape(-1) + b_bn)

    return fusedconv

class ConvBN(torch.nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3, stride=2)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return self.activation(x)
    
    def fuse(self):
        return fuse_conv_and_bn(self.conv, self.bn)
    

class Head(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.module = nn.Sequential(
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10)
            # nn.Softmax()
        )

    def forward(self, x):
        return self.module(x)

class ComposedClassifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.convbn1 = ConvBN(1, 8)
        self.convbn2 = ConvBN(8, 8*8)
        self.head = Head()
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.convbn1(x)
        x = self.convbn2(x)
        y = self.head(x)
        y = self.dequant(y)
        return y
    

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    correct = 0
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            # print(correct)
            # print((len(y) * 100))
            # print(correct * 100. / (len(y) * 100))
            correct = 0


def test(dataloader, model, loss_fn, cuda=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            if cuda:
                X, y = X.to('cuda'), y.to('cuda')
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = ComposedClassifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [4]:
model = model.to("cuda")
model.eval()
backend = "x86"
model.qconfig = torch.quantization.get_default_qat_qconfig(backend)

# model_fp32_fused = torch.ao.quantization.fuse_modules(model,
#     ['convbn1', 'convbn2'])

model_fp32_prepared = torch.quantization.prepare_qat(model.train(), inplace=False)
optimizer = torch.optim.SGD(model_fp32_prepared.parameters(), lr=1e-3)

for i in range(20):
    model_fp32_prepared = model_fp32_prepared.to('cuda')
    train(train_dataloader, model_fp32_prepared, loss_fn, optimizer)
    model_fp32_prepared = model_fp32_prepared.to('cpu')
    test(test_dataloader, model_fp32_prepared, loss_fn)

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/fake_quantize.py:353: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/ReduceAllOps.cpp:71.)
  return torch.fused_moving_avg_obs_fake_quant(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/fake_quantize.py:353: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:687.)
  retur

Test Error: 
 Accuracy: 47.2%, Avg loss: 1.979681 

Test Error: 
 Accuracy: 52.4%, Avg loss: 1.701463 

Test Error: 
 Accuracy: 57.6%, Avg loss: 1.480321 

Test Error: 
 Accuracy: 61.7%, Avg loss: 1.323876 

Test Error: 
 Accuracy: 64.4%, Avg loss: 1.206758 

Test Error: 
 Accuracy: 66.1%, Avg loss: 1.123938 

Test Error: 
 Accuracy: 67.2%, Avg loss: 1.062452 

Test Error: 
 Accuracy: 68.2%, Avg loss: 1.008818 

Test Error: 
 Accuracy: 68.7%, Avg loss: 0.972030 

Test Error: 
 Accuracy: 69.6%, Avg loss: 0.940650 

Test Error: 
 Accuracy: 70.2%, Avg loss: 0.912938 

Test Error: 
 Accuracy: 70.5%, Avg loss: 0.889585 

Test Error: 
 Accuracy: 71.1%, Avg loss: 0.871707 

Test Error: 
 Accuracy: 71.5%, Avg loss: 0.856695 

Test Error: 
 Accuracy: 71.4%, Avg loss: 0.842937 

Test Error: 
 Accuracy: 71.9%, Avg loss: 0.828555 

Test Error: 
 Accuracy: 72.2%, Avg loss: 0.815745 

Test Error: 
 Accuracy: 72.5%, Avg loss: 0.804035 

Test Error: 
 Accuracy: 72.6%, Avg loss: 0.793586 

Test Error: 

In [5]:
model_fp32_prepared.to("cpu")
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared, inplace=False)

In [6]:
%%timeit
test(test_dataloader, model_fp32_prepared, loss_fn)

Test Error: 
 Accuracy: 72.9%, Avg loss: 0.785269 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785155 

Test Error: 
 Accuracy: 72.7%, Avg loss: 0.785301 

Test Error: 
 Accuracy: 72.7%, Avg loss: 0.784912 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.784845 

Test Error: 
 Accuracy: 72.7%, Avg loss: 0.784882 

Test Error: 
 Accuracy: 72.7%, Avg loss: 0.784929 

Test Error: 
 Accuracy: 72.7%, Avg loss: 0.784903 

1.14 s ± 346 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
test(test_dataloader, model_int8, loss_fn)

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785484 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785484 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785484 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785484 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785484 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785484 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785484 

Test Error: 
 Accuracy: 72.8%, Avg loss: 0.785484 

991 ms ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
torch.save(model_int8, "model_int8_qat.pt")

In [9]:
from copy import deepcopy
model = deepcopy(model_fp32_prepared)
model.to('cpu')
model.eval()
model = torch.quantization.convert(model, inplace=False)
print(dir(model))
torch.save(model, "test")

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hoo